In [1]:
#Import packages
from scipy.stats import norm
import matplotlib.pyplot as plt
import seaborn as sns
from math import erf
from BayesEst import exp_val,gaussian_denominator,denominator,bayesian_est
import numpy as np
from scipy.integrate import quad


### Calculating the Bayesian Optimal estimator

We assume that all students have a latent quality $W$, their scores $\hat{W^A},\hat{W^B}$ are respectively the rating scores of college A and B. These ratings scores are the sum of the latent quality and a noise term $\varepsilon$ drawn indepdently at each college, i.e. 
\begin{equation}
   \forall s \in S, \text{ for } C \in \{A,B\}, \widehat{W^C_s} = \varepsilon_s^C + W_s 
\end{equation}
where $S$ is the student set and $s$ is an arbitrary student. Moreover, we assume that the latent qualities of all students are drawn from a (group-indepedent) normal distribution, the noises are also normally distributed and depend on the group:
\begin{equation}
     \forall s \in S, \text{ for } C \in \{A,B\}, W_s \sim \mathcal{N}(0,\chi^2), \varepsilon^C_s \sim \mathcal{N}(0,\sigma^2_{G(s)}).
\end{equation}
$\hat{W}_s^C$ then is a normal distribution with mean $0$ and variance $\chi_{G(s)}^2 = \chi^2 + \sigma^2_{G(s)}$.

In this notebook, we will calculate the conditional expectation $E[W_s|\widehat{W_s^A},\widehat{W_s^B}<P_B]$. 

This conditional expectation can be rewritten as

$$E[W_s|\widehat{W_s^A},\widehat{W_s^B}<P_B] = \int_R w \frac{f_{W_s,\widehat{W_s^A},\widehat{W_s^B} < P_B}(w,w_A,w_B < P_B)}{f_{\widehat{W_s^A},\widehat{W_s^B} < P_B}(w_A,w_B < P_B)} dw$$

## New way to calculate the conditional expectation

Use the same settings,
the conditional expectation can be rewritten as

$$E[W_s|\widehat{W_s^A},\widehat{W_s^B}<P_B] = \int_R w \frac{f_{W_s,\widehat{W_s^A},\widehat{W_s^B} < P_B}(w,w_A,w_B < P_B)}{f_{\widehat{W_s^A},\widehat{W_s^B} < P_B}(w_A,w_B < P_B)} dw$$

$$
\begin{align}
    &\int_R w \Phi(\frac{P_B - w}{\sigma}) \phi(\frac{w-a}{b})dw \nonumber\\
    &=\int_R w \Phi(\frac{P_B - w}{\sigma}) \frac{1}{\sqrt{2 \pi}} \exp{\left(-\frac{\left(\frac{w-a}{b}\right)^2}{2}\right)}dw \nonumber\\
    &= \int_R w \Phi(\frac{P_B - w}{\sigma}) \frac{1}{\sqrt{2 \pi}} \exp{\left(-\frac{\left(\frac{w-a}{b}\right)^2}{2}\right)}dw \nonumber\\
    &=\frac{1}{\sqrt{2 \pi}} \int_R w \Phi(\frac{P_B - w}{\sigma}) \exp{\left(-\frac{\left(\frac{w-a}{b}\right)^2}{2}\right)}dw \nonumber\\
    &= \frac{1}{\sqrt{2 \pi}} \int_R w \frac{1}{2}(1+\text{erf}(\frac{P_B - w}{\sigma \sqrt{2}})) \exp{\left(-\frac{\left(\frac{w-a}{b}\right)^2}{2}\right)}dw \nonumber\\
    &= \frac{1}{\sqrt{2 \pi}} \int_R w \frac{1}{2} \exp{\left(-\frac{\left(\frac{w-a}{b}\right)^2}{2}\right)}dw + \frac{1}{\sqrt{2 \pi}} \int_R w \frac{1}{2}\text{erf}(\frac{P_B - w}{\sigma \sqrt{2}}) \exp{\left(-\frac{\left(\frac{w-a}{b}\right)^2}{2}\right)}dw
\end{align}$$

The following defined functions are to calculate the last part of the equation numerically and will be compared to the original Bayes_est function.

In [2]:
#Define function inside the integral
def integrand_test_1(w,w_a,p_b,chi,sigma):
    a = (chi**2/(chi**2+sigma**2))*w_a
    b = np.sqrt(chi**2*sigma**2/(chi**2+sigma**2))
    return w*np.exp(-((w-a)/b)**2/2)
#Calculate the integral with respect to w
def exp_val_new_1(w_a,p_b,chi,sigma):
    return quad(integrand_test_1, -np.inf,np.inf,args=(w_a,p_b,chi,sigma))
def integrad_test_2(w,w_a,p_b,chi,sigma):
    a = chi**2/(chi**2+sigma**2)*w_a
    b = np.sqrt(chi**2*sigma**2/(chi**2+sigma**2))
    return w*np.exp(-((w-a)/b)**2/2)*erf((p_b-w)/(np.sqrt(2)*sigma))
def exp_val_new_2(w_a,p_b,chi,sigma):
    return quad(integrad_test_2, -np.inf,np.inf,args=(w_a,p_b,chi,sigma))
def test_numerator(w_a,p_b,chi,sigma):
    b = np.sqrt(chi**2*sigma**2/(chi**2+sigma**2))
    return (1/(2*b*np.sqrt(2*np.pi)))*(exp_val_new_1(w_a,p_b,chi,sigma)[0] + exp_val_new_2(w_a,p_b,chi,sigma)[0])

In [3]:
def original_numerator_integrand(w,w_a,p_b,chi,sigma):
    a = (chi**2/(chi**2+sigma**2))*w_a
    b = np.sqrt(chi**2*sigma**2/(chi**2+sigma**2))
    return w*norm.cdf(p_b -w,0,sigma)*norm.pdf((w-a),0,b)
def orig_exp_val(w_a,p_b,chi,sigma):
    return quad(original_numerator_integrand,-np.inf,np.inf,args=(w_a,p_b,chi,sigma))

In [4]:
def original_numerator_integrand_new(w,w_a,p_b,chi,sigma):
        return w*norm.cdf((p_b -w),0,sigma)*norm.pdf((w_a-w),0,sigma)
def orig_exp_val_new(w_a,p_b,chi,sigma):
    return quad(original_numerator_integrand,-np.inf,np.inf,args=(w_a,p_b,chi,sigma))

In [5]:
w_a = 4
P_B = 5
chi = 4
sigma = 2

In [6]:
test_numerator(w_a,P_B,chi,sigma)*norm.pdf(w_a,0,np.sqrt(sigma**2+chi**2))

np.float64(0.12057150244086456)

In [7]:
orig_exp_val_new(w_a,P_B,chi,sigma)[0]*norm.pdf(w_a,0,np.sqrt(chi**2+sigma**2))

np.float64(0.12057150244086456)

In [8]:
orig_exp_val(w_a,P_B,chi,sigma)[0]*norm.pdf(w_a,0,np.sqrt(chi**2+sigma**2))

np.float64(0.12057150244086456)

In [9]:
exp_val(w_a,P_B,chi,sigma)

(0.12057150244086448, 2.4600658611068408e-09)

In [10]:
print(f'The value of the expected value using the above calculation is {test_numerator(w_a,P_B,chi,sigma)/denominator(w_a,P_B,chi,sigma)}')
print(f'The value of the expected value using the original calculation is {bayesian_est(w_a,P_B,chi,sigma)}')
#print(f'The value of the expected value using the analytical formula is {anal_bayes(w_a,P_B,chi,sigma)}')

The value of the expected value using the above calculation is 2.6926673189306616
The value of the expected value using the original calculation is 2.6926673189306594


Moreover, for the first part of the last function above, let $u = \frac{w-a}{b}$, we can rewrite 
$$\begin{align}
    \frac{1}{\sqrt{2 \pi}} \int_R w \frac{1}{2} \exp{\left(-\frac{\left(\frac{w-a}{b}\right)^2}{2}\right)}dw &= \frac{1}{\sqrt{2 \pi}} \int_R (ub+a) \frac{1}{2} \exp({-\frac{u^2}{2})}bdu \nonumber \\
    &= \frac{b^2}{2} \frac{1}{\sqrt{2 \pi}} \int_R u  \exp({-\frac{u^2}{2})}du + \frac{ab}{2}\frac{1}{\sqrt{2 \pi}} \int_R   \exp({-\frac{u^2}{2})}du \nonumber \\
    &= \frac{ab}{2}
\end{align}
$$

In [11]:
#Function to analytically calculate the integral
def anal_exp_val_new_1(w_a,p_b,chi,sigma):
    a = chi**2/(chi**2+sigma**2)*w_a
    b = np.sqrt(chi**2*sigma**2/(chi**2+sigma**2))
    #return a*b/2
    return a/2

In [12]:
a = chi**2/(chi**2+sigma**2)*w_a
b = np.sqrt(chi**2*sigma**2/(chi**2+sigma**2))
print(f'The numerical integration of the above equation is {(1/(b*2*np.sqrt(2*np.pi)))*exp_val_new_1(w_a,P_B,chi,sigma)[0]}')
print(f'The analytically value of the integral above is {anal_exp_val_new_1(w_a,P_B,chi,sigma)}')

The numerical integration of the above equation is 1.5999999999999999
The analytically value of the integral above is 1.6


We now proceed to calculate the value of 
$$\frac{1}{\sqrt{2 \pi}} \int_R w \frac{1}{2}\text{erf}(\frac{P_B - w}{\sigma \sqrt{2}}) \exp{\left(-\frac{\left(\frac{w-a}{b}\right)^2}{2}\right)}dw$$


Let $t = \frac{a - w}{b}$, we can rewrite $w = a - bt, \frac{P_B - w}{\sigma \sqrt{2}} = \frac{P_B + bt - a}{\sigma \sqrt{2}},$ and $dw = -bdt$. The equation above can be rewritten

$$\begin{align}
    &\frac{1}{\sqrt{2 \pi}} \int_R w \frac{1}{2}\text{erf}(\frac{P_B - w}{\sigma \sqrt{2}}) \exp{\left(-\frac{\left(\frac{w-a}{b}\right)^2}{2}\right)}dw \nonumber \\
    &= \frac{1}{\sqrt{2 \pi}} -\int_R (a-bt) \frac{1}{2}\text{erf}(\frac{P_B +bt - a}{\sigma \sqrt{2}}) \exp\frac{-t^2}{2}-bdt \nonumber\\
    &=  \frac{b}{\sqrt{2 \pi}} \int_R (a-bt) \frac{1}{2}\text{erf}(\frac{P_B +bt - a}{\sigma \sqrt{2}}) \exp\frac{-t^2}{2}dt \nonumber\\
    &= \frac{ab}{\sqrt{2 \pi}} \int_R  \frac{1}{2}\text{erf}(\frac{P_B +bt - a}{\sigma \sqrt{2}}) \exp\frac{-t^2}{2}dt - \frac{b^2}{\sqrt{2 \pi}} \int_R t \frac{1}{2}\text{erf}(\frac{P_B +bt - a}{\sigma \sqrt{2}}) \exp\frac{-t^2}{2}dt 
\end{align}$$

In [21]:
print(f'Value of the first equation numerically calculated is {(1/(2*np.sqrt(2*np.pi)))*exp_val_new_2(w_a,P_B,chi,sigma)[0]}')
print(f'Value of the last equation numerically calculated is {exp_val_new_3(w_a,P_B,chi,sigma)[0]*(a*b)/(2*np.sqrt(2*np.pi)) - exp_val_new_4(w_a,P_B,chi,sigma)[0]*(b**2)/(2*np.sqrt(2*np.pi))}')

Value of the first equation numerically calculated is 0.7448017962578701
Value of the last equation numerically calculated is 0.7448017962578695


We first propose the solution to the first part of the above equation

$$\begin{align}
    \frac{ab}{\sqrt{2 \pi}} \int_R  \frac{1}{2}\text{erf}(\frac{P_B +bt - a}{\sigma \sqrt{2}}) \exp\frac{-t^2}{2}dt &= \frac{ab}{2\sqrt{2 \pi}} \sqrt{2\pi} \text{erf} \left( \frac{P_B - a}{ \sqrt{2(\sigma^2 + b^2)}} \right) \nonumber\\
    &= \frac{ab}{2} \text{erf} \left( \frac{P_B - a}{ \sqrt{2(\sigma^2 + b^2)}} \right) 
\end{align}
$$

In [13]:
#Numerical integration of the above equation
def integrand_test_3(t,w_a,p_b,chi,sigma):
    a = chi**2/(chi**2+sigma**2)*w_a
    b = np.sqrt(chi**2*sigma**2/(chi**2+sigma**2))
    return erf((P_B+b*t-a)/(sigma*np.sqrt(2)))*np.exp(-t**2/2)
def exp_val_new_3(w_a,p_b,chi,sigma):
    return quad(integrand_test_3,-np.inf,np.inf,args=(w_a,p_b,chi,sigma))

In [14]:
#Analytically calculating the above equation
def anal_exp_val_new_2_1(w_a,p_b,chi,sigma):
    a = chi**2/(chi**2+sigma**2)*w_a
    b = np.sqrt(chi**2*sigma**2/(chi**2+sigma**2))
    #return a*b/2*erf((P_B-a)/(np.sqrt(2*(sigma**2+b**2))))
    return a/2*erf((P_B-a)/(np.sqrt(2*(sigma**2+b**2))))

In [15]:
a = chi**2/(chi**2+sigma**2)*w_a
b = np.sqrt(chi**2*sigma**2/(chi**2+sigma**2))
print(f'value of the equation using numerical integration is {exp_val_new_3(w_a,P_B,chi,sigma)[0]*(a)/(2*np.sqrt(2*np.pi))}')
print(f'Value of the equation using analytical formula {anal_exp_val_new_2_1(w_a,P_B,chi,sigma)}')

value of the equation using numerical integration is 0.7962640730231966
Value of the equation using analytical formula 0.7962640730231967


The solution to the second part of the equation is

$$\begin{align}
    \frac{b^2}{\sqrt{2 \pi}} \int_R t \frac{1}{2}\text{erf}(\frac{P_B +bt - a}{\sigma \sqrt{2}}) \exp\frac{-t^2}{2}dt &= \frac{b^2}{\sqrt{2 \pi}}  \frac{\frac{b}{\sigma \sqrt{2}}}{\frac{1}{2} \sqrt{(\frac{b}{\sigma \sqrt{2}})^2+\frac{1}{2}}}\exp \left( - \frac{\frac{1}{2} \beta^2 }{(\frac{b}{\sigma \sqrt{2}})^2 + \frac{1}{2}} \right) \nonumber \\
    &= \frac{b^2}{\sqrt{2 \pi}} \frac{2b}{\sqrt{b^2+\sigma^2}}\exp \left( - \frac{\frac{1}{2} (\frac{P_B - a}{\sigma \sqrt{2}})^2 }{\frac{b^2 + \sigma^2}{2 \sigma^2}} \right) \nonumber \\
    &=  \frac{2b^3}{\sqrt{2\pi(b^2+\sigma^2) }}\exp \left( - \frac{ (P_B - a)^2}{2(b^2 + \sigma^2)} \right) \nonumber \\
\end{align}
$$

In [16]:
#Numerical integration of the above equation
def integrand_test_4(t,w_a,p_b,chi,sigma):
    a = chi**2/(chi**2+sigma**2)*w_a
    b = np.sqrt(chi**2*sigma**2/(chi**2+sigma**2))
    return t* erf((p_b+b*t-a)/(sigma*np.sqrt(2)))*np.exp(-t**2/2)
def exp_val_new_4(w_a,p_b,chi,sigma):
    return quad(integrand_test_4,-np.inf,np.inf,args=(w_a,p_b,chi,sigma))

In [17]:
#Analytically calculating the above equation
def anal_exp_val_new_2_2(w_a,p_b,chi,sigma):
    a = chi**2/(chi**2+sigma**2)*w_a
    b = np.sqrt(chi**2*sigma**2/(chi**2+sigma**2))
    return (b**2)/(np.sqrt(2*np.pi*(b**2 + sigma**2)))*np.exp(-(P_B-a)**2/(2*(b**2+sigma**2)))

In [18]:
w_a = 5
P_B = 1
chi= 4
sigma = 2

In [19]:
a = chi**2/(chi**2+sigma**2)*w_a
b = np.sqrt(chi**2*sigma**2/(chi**2+sigma**2))
print(f'Value of the equation numerically calculated is {exp_val_new_4(w_a,P_B,chi,sigma)[0]*(b)/(2*np.sqrt(2*np.pi))}')
print(f'Value of the equation analytically calculated is {anal_exp_val_new_2_2(w_a,P_B,chi,sigma)}')

Value of the equation numerically calculated is 0.25465941948456333
Value of the equation analytically calculated is 0.25465941948456344


The denominator follows a Gaussian distribution with mean $\frac{\chi^2}{\chi^2 + \sigma^2}\widehat{W^A_s}$ and variance $\frac{\sigma^4 + 2\sigma^2\chi^2}{\chi^2 + \sigma^2}$. The denominator can simply be calculated using the CDF of normal distribution.


In [28]:
def anal_bayes(w_a,p_b,chi,sigma):
    a = chi**2/(chi**2+sigma**2)*w_a
    b = np.sqrt(chi**2*sigma**2/(chi**2+sigma**2))
    return (anal_exp_val_new_1(w_a,P_B,chi,sigma) + anal_exp_val_new_2_1(w_a,p_b,chi,sigma)- anal_exp_val_new_2_2(w_a,p_b,chi,sigma))/denominator(w_a,p_b,chi,sigma) 

In [29]:
anal_bayes(w_a,4,4,2)

np.float64(0.544891070162764)

In [30]:
anal_cond_exp(w_a,4,4,2)

np.float64(3.0484671380518553)

In [31]:
bayesian_est(w_a,4,4,2)

np.float64(3.048467138051853)